## Write to and read from a Delta Lake table

### Write a Spark DataFrame to a Delta Lake table

In [ ]:
from pyspark.sql import SparkSession
import subprocess

spark = SparkSession.builder \
    .appName("DeltaLakeDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

delta_path = "/opt/spark/delta-tables/"

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/17 18:01:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/17 18:01:32 WARN SparkSession: Cannot use io.delta.sql.DeltaSparkSessionExtension to configure session extensions.
java.lang.ClassNotFoundException: io.delta.sql.DeltaSparkSessionExtension
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:593)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:526)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:536)
	at java.base/java.lang.Class.forName(Class.java:515)
	at org.apache.spark.util.SparkClassUtils.classForName(SparkClassUtils.scala:42)

In [2]:
data = spark.range(0, 5)

(data
  .write
  .format("delta")
  .mode("overwrite")
  .save(delta_path + "table1")
)

Py4JJavaError: An error occurred while calling o36.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Make sure the provider name is correct and the package is properly registered and compatible with your Spark version. SQLSTATE: 42K02
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:764)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:686)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:745)
	at org.apache.spark.sql.classic.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:596)
	at org.apache.spark.sql.classic.DataFrameWriter.saveCommand(DataFrameWriter.scala:141)
	at org.apache.spark.sql.classic.DataFrameWriter.save(DataFrameWriter.scala:115)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMeth

### Read the above Delta Lake table to a Spark DataFrame and display the DataFrame

In [ ]:
df = (spark
        .read
        .format("delta")
        .load(delta_path + "table1")
        .orderBy("id")
      )

df.show()

## Overwrite a Delta Lake table

### Overwrite the Delta Lake table written in the above step

In [ ]:
data = spark.range(5, 10)

view_data = data.createOrReplaceTempView("data_view")
table2 = delta_path + "table1"
spark.sql(f"""
  MERGE INTO delta.`{table2}` AS target
  USING data_view AS source
  ON target.id = source.id
  WHEN NOT MATCHED THEN
    INSERT *
""")

### Read the above overwritten Delta Lake table to a Spark DataFrame and display the DataFrame

In [ ]:
df = (spark
        .read
        .format("delta")
        .load(delta_path + "table1")
        .orderBy("id")
      )

df.show()

## Delta Lake and [ACID](https://en.wikipedia.org/wiki/ACID)

### Update Delta Lake Table

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *


delta_table = DeltaTable.forPath(spark, delta_path + "table1")

spark.sql(f"""
  UPDATE delta.`{delta_path}table1`
  SET id = id + 100
  WHERE id % 2 == 0
""")


In [ ]:
spark.sql("""
vacuum delta.`/opt/spark/delta-tables/table1`
""")

In [ ]:
spark.sql("""
        optimize delta.`/opt/spark/delta-tables/table1`
        """)

In [ ]:
spark.sql(f"""
    ALTER TABLE delta.`{delta_path}table1`
    SET TBLPROPERTIES (
        'delta.enableChangeDataCapture' = 'true'
    )
""")

In [ ]:
#delta_table.toDF().orderBy("id").show()
delta_table.toDF().createOrReplaceTempView("delta_table_view")
spark.sql("""
  select * from table_view
  order by id
""").show()

In [ ]:
spark.sql(f"""
  UPDATE delta.`{delta_path}table2`
  SET id = id + 100
  WHERE id % 2 == 0
""")

###  `delete`

In [ ]:
# Delete every even value
(delta_table
  .delete(
    condition = expr("id % 2 == 0")
  )
)

(delta_table
  .toDF()
  .orderBy("id")
  .show()
)

### `merge` Delta Lake Table

In [ ]:
# Upsert (merge) new data
new_data = spark.range(0, 20)

(delta_table.alias("old_data")
  .merge(
      new_data.alias("new_data"),
      "old_data.id = new_data.id"
      )
  .whenMatchedUpdate(set = { "id": col("new_data.id") })
  .whenNotMatchedInsert(values = { "id": col("new_data.id") })
  .execute()
)

(delta_table
  .toDF()
  .orderBy("id")
  .show()
)

## Time travel feature of Delta Lake

### Display the entire history of the above Delta Lake table

In [ ]:
# get the full history of the table
delta_table_history = (DeltaTable
                        .forPath(spark, f"{delta_path}delta-table")
                        .history()
                      )

(delta_table_history
   .select("version", "timestamp", "operation", "operationParameters", "operationMetrics", "engineInfo")
   .show()
)

### Latest version of the Delta Lake table

In [ ]:
# get the full history of the table
delta_table_history = (DeltaTable
                        .forPath(spark, "/tmp/delta-table")
                        .history()
                      )

(delta_table_history
   .select("version", "timestamp", "operation", "operationParameters", "operationMetrics", "engineInfo")
   .show()
)

### Latest version of the Delta Lake table

In [ ]:
df = (spark
        .read
        .format("delta")
        .load("/tmp/delta-table")
        .orderBy("id")
      )

df.show()

### Time travel to the version `0` of the Delta Lake table using Delta Lake's history feature

In [ ]:
df = (spark
        .read
        .format("delta")
        .option("versionAsOf", 0) # we pass an option `versionAsOf` with the required version number we are interested in
        .load("/tmp/delta-table")
        .orderBy("id")
      )

df.show()

### Time travel to the version `3` of the Delta Lake table using Delta Lake's  history feature

In [ ]:
df = (spark
        .read
        .format("delta")
        .option("versionAsOf", 3) # we pass an option `versionAsOf` with the required version number we are interested in
        .load("/tmp/delta-table")
        .orderBy("id")
      )

df.show()